In [239]:
import pandas as pd
import re
import codecs

# Data

In [240]:
path = ".\Data\PoetryFoundationData.csv"
df = pd.read_csv(path)
df.columns

Index(['Unnamed: 0', 'Title', 'Poem', 'Poet', 'Tags'], dtype='object')

In [241]:
df_red = df[["Title", "Poem", "Poet"]]


In [252]:
df2 = df_red.groupby(['Poet'])['Poet'].count().nlargest(100)

In [253]:
df2

Poet
Edna St. Vincent Millay    10
Ntozake Shange             10
Quan Barry                 10
Thomas Centolella          10
Kahlil Gibran               9
                           ..
Bei Dao                     3
Bill Zavatsky               3
Charles Olson               3
Chinua Achebe               3
Christian Wiman             3
Name: Poet, Length: 100, dtype: int64

In [243]:
# cleaning
df_red.loc[:, "Poem"] = df_red.loc[:, "Poem"].str.replace("\r","")
df_red.loc[:, "Poem"] = df_red.loc[:, "Poem"].str.rstrip("\n")

df_red.loc[:, "Title"] = df_red.loc[:, "Title"].str.replace(r" {2,}", "", regex=True)
df_red.loc[:, "Title"] = df_red.loc[:, "Title"].str.replace("\n", "")
df_red.loc[:, "Title"] = df_red.loc[:, "Title"].str.replace("\r", "")

df_red.loc[:, "Poet"] = df_red.loc[:, "Poet"].str.replace("\n", "")
df_red.loc[:, "Poet"] = df_red.loc[:, "Poet"].str.replace("\r", "")

In [244]:
nan_poems = df_red["Poem"].isnull().sum()
nan_poets = df_red["Poet"].isnull().sum()
nan_title = df_red["Title"].isnull().sum()
print(f"Num NaN Poems: {nan_poems}" +
      f"\nNum NaN Poets: {nan_poets}" +
      f"\nNum NaN Title: {nan_title}")

Num NaN Poems: 0
Num NaN Poets: 0
Num NaN Title: 0


In [245]:
"""
Poem text files should be formatted as:
TITLE
AUTHOR
TEXTTEXTTEXT[...]
******
TITLE2
AUTHOR(2)
TEXT...
"""

with codecs.open(".\Data\style_input.txt", "w", "utf-8") as f:
    aux = ""
    for _, row in df_red.iterrows():
        new_line = row["Title"] + "\n" +\
                    row["Poet"] + "\n" +\
                    row["Poem"] + "\n******\n"

        f.write(new_line)
      

In [256]:
!pip install -q selenium

In [277]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


In [330]:
driver = webdriver.Chrome()
driver.get("https://poets.org/poet/john-wieners")  
import time, random
"""time.sleep(1)

mytable = driver.find_element(By.CSS_SELECTOR, 'tbody')
i =0
for row in mytable.find_elements(By.CSS_SELECTOR, 'td'):
    link= mytable.find_elements(By.TAG_NAME, "a")
    link[2].click()
    time.sleep(1)
    i+=1
    if i>2:
        break
"""
def get_elements(web):
    driver.get(web)
    time.sleep(2) 
    mytable = driver.find_element(By.CSS_SELECTOR, 'tbody')
    return mytable.find_elements(By.TAG_NAME, "a")

num_elems = len(get_elements("https://poets.org/poet/john-wieners"))

i = 0
while i<num_elems:
    elements = get_elements("https://poets.org/poet/john-wieners")
    
    link = elements[i]
    print(link.text)
    link.click()
    time.sleep(random.randint(2, 5))

    i+=1

10
In Public
Billie


KeyboardInterrupt: 

: 

In [297]:
elements

[]